In [1]:
import requests
import pandas as pd
import lxml.html as lh

print("Libraries imported!")

Libraries imported!


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website
webcontent = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = webcontent.xpath('//tr')

In [3]:
#Create empty list
postal_code_list=[]

#For each row, store each first element (header) and an empty list
for elem in tr_elements[0]:
    header_name=elem.text_content()
    header_name=header_name.replace('\n','')
    postal_code_list.append((header_name,[]))

In [4]:
#Since out first row is the header, data is stored on the second row onwards
for idx in range(1,len(tr_elements)):
    
    tr_element = tr_elements[idx]
    
    #bypass if no.of column not equal to 3
    if len(tr_element)!=3:
        break
    
    col_idx=0
    for elem in tr_element.iterchildren():
        data=elem.text_content()
        data=data.replace('\n','')
        postal_code_list[col_idx][1].append(data)
        col_idx+=1

In [5]:
#convert to dataframe
Dict={title:column for (title,column) in postal_code_list}
df=pd.DataFrame(Dict)

#drop rows with invalid values
not_assigned_borough_idx = df[ df['Borough'] == 'Not assigned' ].index
invalid_postal_code_idx = df[ df['Postal Code'] == '' ].index

df.drop(not_assigned_borough_idx , inplace=True)
df.drop(invalid_postal_code_idx , inplace=True)

#reset index
df.reset_index(drop=True)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
df.shape

(103, 3)